In [44]:
import sys
import os
parent_folder_path = os.path.abspath(os.path.join(os.getcwd(), "..", "utils"))
sys.path.append(parent_folder_path)
from utils import arrify_string_columns
data_folder = os.path.join("..", "data")

import pandas as pd
import matplotlib.pyplot as plt

In [45]:
original_df = pd.read_csv(os.path.join(data_folder, "time_sorted_table.csv"), sep=';', index_col=0)
arrify_string_columns(original_df)

In [46]:
array_cols = ['seconds_to_incident_sequence', 'vehicles_sequence', 'events_sequence',
                    'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence']

df = original_df[array_cols]

In [47]:
df[['both_true', 'both_false']] = df.apply(
    lambda row: pd.Series({
        'both_true': [r1 and r2 for r1, r2 in zip(row.dj_ac_state_sequence, row.dj_dc_state_sequence)],
        'both_false': [not r1 and not r2 for r1, r2 in zip(row.dj_ac_state_sequence, row.dj_dc_state_sequence)]
    }), 
    axis=1
)

/tmp/ipykernel_149585/427967969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['both_true', 'both_false']] = df.apply(
/tmp/ipykernel_149585/427967969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['both_true', 'both_false']] = df.apply(


In [48]:
# It never happens that both states are true
df[df['both_true'].apply(lambda x: any(x))]

,seconds_to_incident_sequence,vehicles_sequence,events_sequence,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,both_true,both_false


TypeError: unhashable type: 'list'